#### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

#### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

#### Важно:

* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

#### Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
* сортировка просмотренных id по популярности (частота появления в просмотренных),
* сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

#### Дополнительные вопросы

Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [2]:
import pandas as pd
import numpy as np

In [3]:
#построим частоты появления id в просмотренных и купленных, создадим лист просмотренных и купленных товаров
# для файла с тренировочными данными
data_train_seen = {}
data_train_bought = {}
data_train =  []
with open("coursera_sessions_train.txt") as infile:
    for line in infile:
        seen = []
        bought = []
        splitted_line = line.split(sep = ';')
        if splitted_line[0] != '':
            for item in splitted_line[0].split(sep = ','):
                seen.append(int(item))
                if int(item) in data_train_seen:
                    data_train_seen[int(item)] += 1
                else:
                    data_train_seen[int(item)] = 1
        if splitted_line[1] != '\n':
            for item in splitted_line[1].split(sep = ','):
                bought.append(int(item))
                if int(item) in data_train_bought:
                    data_train_bought[int(item)] += 1
                else:
                    data_train_bought[int(item)] = 1
        data_train.append([seen, bought])

In [4]:
#построим частоты появления id в просмотренных и купленных, создадим лист просмотренных и купленных товаров
# для файла с тестовыми данными
data_test =  []
with open("coursera_sessions_test.txt") as infile:
    for line in infile:     
        seen = []
        bought = []
        splitted_line = line.split(sep = ';')
        if splitted_line[0] != '':
            for item in splitted_line[0].split(sep = ','):
                seen.append(int(item))
        if splitted_line[1] != '\n':
            for item in splitted_line[1].split(sep = ','):
                bought.append(int(item))
        data_test.append([seen, bought])

In [5]:
data_train_seen_df = pd.DataFrame(data = data_train_seen.items(), columns = ["Id", "Count"])
data_train_seen_df

,Id,Count
0,0,6
1,1,6
2,2,9
3,3,7
4,4,11
...,...,...
77059,102803,1
77060,102804,1
77061,102805,1
77062,102806,1


In [128]:
data_train_bought_df = pd.DataFrame(data = data_train_bought.items(), columns = ["Id", "Count"])
data_train_bought_df

,Id,Count
0,67,1
1,60,1
2,63,1
3,86,2
4,199,1
...,...,...
4474,50751,1
4475,5895,1
4476,25935,1
4477,2807,1


In [206]:
#реализуем рекомендательный алгоритм сортировкой просмотренных id по популярности
data_train_seen_df = data_train_seen_df.sort_values(by = "Count", ascending = False)
data_train_seen_df[data_train_seen_df.Id == 86].Id

86

In [190]:
data_test[0][0]

[6, 7, 8]

In [200]:
def recommend(data, data_freq_df, k): 
    recs = []
    for seen, bought in data:
        if bought != []:
            temp = []
            k = min(k, len(seen))
            for item in seen:
                if item in data_freq_df.Id:
                    temp.append([data_freq_df[data_freq_df.Id == item].Count.item(),
                                 data_freq_df[data_freq_df.Id == item].index.item(),
                                 data_freq_df[data_freq_df.Id == item].Id.item()])
                else:
                    temp.append([0, -1, item])    
            temp = sorted(temp, key=lambda x: (x[0], x[1]))
            temp.reverse()
            rec = []
            for i in range(0, k):
                rec.append(temp[i][2])
            recs.append(rec)
    return recs

In [214]:
def calculate_stat(data, recs, k):
    precisions = []
    recalls = []
    it = 0
    for i in range(0, len(data)):
        numerator = 0
        bought = data[i][1]
        if bought != []:
            for j in range(0, len(bought)):
                if bought[j] in recs[it]:
                    numerator += 1
            precisions.append(numerator/k)
            recalls.append(numerator/len(bought))
            it += 1
    return (sum(recalls)/len(recalls), sum(precisions)/len(precisions))

In [208]:
train_recs = recommend(data_train, data_train_seen_df, 1)

In [209]:
train_recs[1]

[85]

In [215]:
stats = calculate_stat(data_train, train_recs, 1)
stats

(0.44161805643235763, 0.510809312638581)

In [216]:
train_recs2 = recommend(data_train, data_train_seen_df, 5)
train_recs2[:2]

[[63, 64, 68, 67, 67], [85, 93, 89, 90, 92]]

In [217]:
stats2 = calculate_stat(data_train, train_recs2, 5)
stats2

(0.4419876055823945, 0.10227272727272384)

In [202]:
train_recs3 = recommend(data_test, data_train_seen_df, 1)

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
stats3 = calculate_stat(data_test, train_recs3)
stats3

In [ ]:
train_recs4 = recommend(data_test, data_train_seen_df, 5, sorting = 'purchasing')
train_recs4[:5]

In [ ]:
stats4 = calculate_stat(data_test, train_recs4)
stats4

In [156]:
a = sorted([9, 3, 0, 1])
a

[0, 1, 3, 9]

In [157]:
a.reverse()

In [158]:
a

[9, 3, 1, 0]

In [198]:
data_train[:5]

[[[0, 1, 2, 3, 4, 5], []],
 [[9, 10, 11, 9, 11, 12, 9, 11], []],
 [[16, 17, 18, 19, 20, 21], []],
 [[24, 25, 26, 27, 24], []],
 [[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39], []]]

In [199]:
data_test[:5]

[[[6, 7, 8], []],
 [[13, 14, 15], []],
 [[22, 23], []],
 [[28, 29, 30, 31, 32, 33], []],
 [[40, 41], []]]